In [1]:
#### add a comment.

%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Prepare Batch

In [ ]:
import os
import torch
import pandas as pd
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
l

data/ProcData/TensorFolder/Task2YearXXX


['A1C@DT-DTDftGrn',
 'A1C@V-A1CNumeDftGrn',
 'Diag@DT-DTDftGrn',
 'Diag@Value-DiagDftGrn',
 'EC@BasicInfo-BasicDftGrn',
 'EC@DT_min-DTDftGrn',
 'P@age-AgeNumeDftGrn',
 'P@basicInfo-basicInfoDftGrn',
 'PN@DT-DTDftGrn',
 'PNSect@SectName-PNSctNmDftGrn',
 'PNSectSent@Sentence-Tk@TknzLLMGrn',
 'Smoking@DT-DTDftGrn',
 'Smoking@V-SmokingDftGrn']

In [ ]:
recfldgrn_list =['P@age-AgeNumeDftGrn',
                 'P@basicInfo-basicInfoDftGrn',
                 
                 'EC@BasicInfo-BasicDftGrn',
                 'EC@DT_min-DTDftGrn',
                 
                 'A1C@DT-DTDftGrn',
                 'A1C@V-A1CNumeDftGrn',
                 
                 'Diag@DT-DTDftGrn',
                 'Diag@Value-DiagDftGrn',
                 
                 'PNSectSent@Sentence-Tk@TknzLLMGrn']

In [ ]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

B-P@age-AgeNumeDftGrn_wgt (4, 19)
B-P@basicInfo-basicInfoDftGrn_idx (4, 2)
B-P-EC@BasicInfo-BasicDftGrn_idx (4, 25, 2)
B-P-EC@DT_min-DTDftGrn_idx (4, 25, 7)
B-P-EC-A1C@DT-DTDftGrn_idx (4, 25, 1, 7)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


In [ ]:
from fieldnn.sublayer.expander import Expander_Layer
from fieldnn.configfn.expanderfn import get_expander_para

full_recfldgrn_list = [
                  i for i in batch_rfg
                ]
full_recfldgrn_list

['B-P@age-AgeNumeDftGrn_wgt',
 'B-P@basicInfo-basicInfoDftGrn_idx',
 'B-P-EC@BasicInfo-BasicDftGrn_idx',
 'B-P-EC@DT_min-DTDftGrn_idx',
 'B-P-EC-A1C@DT-DTDftGrn_idx',
 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [ ]:
full_rfg_to_basicInfo = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) full recfldgrn information
    recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
    rec, fld = recfld.split('@')
    grn_suffix = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
    grn, suffix = grn_suffix.split('_')
    prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
    recfldgrn = rec + '@' + fld + '-' + grn

    # (2) get vocab information
    fldgrn_folder = 'data/ProcData/FldGrnInfo'
    fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
    df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)

    # no matter what type of grain, in the end, we will have vocab_tokenizer.
    if 'LLM' in full_recfldgrn:
        vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        # vocab_size = len(vocab_tokenizer)
        # print(vocab_size)
        init = vocab_tokenizer.name_or_path
    else:
        vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        # vocab_size = len(vocab_tokenizer)
        # print(vocab_size)
        init = 'random'
    
    d = {'vocabsize_tokenizer': vocab_tokenizer, 'init': init}
    full_rfg_to_basicInfo[full_recfldgrn] = d
    
    
len(full_rfg_to_basicInfo)

9

In [ ]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


full_rfg_to_ExpanderConfig = {}

for full_recfldgrn in full_recfldgrn_list:
    
    # determine the nn_name
    if '_idx' in full_recfldgrn and 'LLM' not in full_recfldgrn:
        nn_name = 'CateEmbed'

    elif '_idx' in full_recfldgrn and 'LLM' in full_recfldgrn:
        nn_name = 'LLMEmbed'

    elif '_wgt' in full_recfldgrn:
        nn_name = 'NumeEmbed'
    else:
        raise ValueError(f'full_recfldgrn is not correct "{full_recfldgrn}"')
    
    d = full_rfg_to_basicInfo[full_recfldgrn]
    
    # print('\n----')
    # print(full_recfldgrn)
    # print(d)
    # print(nn_name)
    expander_para = get_expander_para(full_recfldgrn, 
                                       nn_name,
                                       embed_size, d['vocabsize_tokenizer'], d['init'], 
                                       process)
    # print(expander_para)
    output_recfld = full_recfldgrn.split('Grn')[0]
    
    full_rfg_to_ExpanderConfig[full_recfldgrn] = {
        'full_recfldgrn': full_recfldgrn,
        'output_recfld': output_recfld,
        'expander_para': expander_para,
    }
    
    
# full_rfg_to_ExpanderConfig

In [ ]:
FullRFG_2_Embed = {}

for full_recfldgrn, ExpanderConfig in full_rfg_to_ExpanderConfig.items():
    
    FullRFG_2_Embed[full_recfldgrn] = Expander_Layer(**ExpanderConfig)
    print(full_recfldgrn)
    print(FullRFG_2_Embed[full_recfldgrn])

B-P@age-AgeNumeDftGrn_wgt
Expander_Layer(
  (Embed): NumeEmbeddingLayer(
    (embedding): Embedding(19, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P@basicInfo-basicInfoDftGrn_idx
Expander_Layer(
  (Embed): CateEmbeddingLayer(
    (embedding): Embedding(7, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P-EC@BasicInfo-BasicDftGrn_idx
Expander_Layer(
  (Embed): CateEmbeddingLayer(
    (embedding): Embedding(9, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P-EC@DT_min-DTDftGrn_i

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
Expander_Layer(
  (Embed): LLMEmbeddingLayer(
    (LLM): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense):

In [ ]:
for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
    print(full_recfldgrn)
    print(info_raw.shape)

    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    print(holder.shape, '<------ holder shape')
    if type(holder_wgt) == str: 
        print(holder_wgt, '<------ holder wgt information')
    else:
        print(holder_wgt.shape, '<------ holder wgt information')

    print('\n')

B-P@age-AgeNumeDftGrn_wgt
(4, 19)
torch.Size([4, 19]) <------ holder shape
torch.Size([4, 19]) <------ holder wgt information


B-P@basicInfo-basicInfoDftGrn_idx
(4, 2)
torch.Size([4, 2]) <------ holder shape
Empty <------ holder wgt information


B-P-EC@BasicInfo-BasicDftGrn_idx
(4, 25, 2)
torch.Size([4, 25, 2]) <------ holder shape
Empty <------ holder wgt information


B-P-EC@DT_min-DTDftGrn_idx
(4, 25, 7)
torch.Size([4, 25, 7]) <------ holder shape
Empty <------ holder wgt information


B-P-EC-A1C@DT-DTDftGrn_idx
(4, 25, 1, 7)
torch.Size([4, 25, 1, 7]) <------ holder shape
Empty <------ holder wgt information


B-P-EC-A1C@V-A1CNumeDftGrn_wgt
(4, 25, 1, 37)
torch.Size([4, 25, 1, 37]) <------ holder shape
torch.Size([4, 25, 1, 37]) <------ holder wgt information


B-P-EC-Diag@DT-DTDftGrn_idx
(4, 25, 22, 7)
torch.Size([4, 25, 22, 7]) <------ holder shape
Empty <------ holder wgt information


B-P-EC-Diag@Value-DiagDftGrn_idx
(4, 25, 22, 3)
torch.Size([4, 25, 22, 3]) <------ holder sha

In [ ]:
full_recfldgrn_EmbedTensor = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
    print('Input:',full_recfldgrn)
    print(info_raw.shape)

    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    print(holder.shape, '<------ holder shape')
    if type(holder_wgt) == str:  
        print(holder_wgt, '<------ holder wgt information')
    else:
        print(holder_wgt.shape, '<------ holder wgt information')
    
    NN = FullRFG_2_Embed[full_recfldgrn] 
    print(holder.shape, '<----- holder shape')
    output_recfld, holder, info = NN(full_recfldgrn, holder, holder_wgt)
    print('Output:', output_recfld)
    print(info.shape, '<----- info shape')
    full_recfldgrn_EmbedTensor[output_recfld] = {'holder': holder, 'info': info}
    print('\n')

Input: B-P@age-AgeNumeDftGrn_wgt
(4, 19)
torch.Size([4, 19]) <------ holder shape
torch.Size([4, 19]) <------ holder wgt information
torch.Size([4, 19]) <----- holder shape
Output: B-P@age-AgeNumeDft
torch.Size([4, 19, 128]) <----- info shape


Input: B-P@basicInfo-basicInfoDftGrn_idx
(4, 2)
torch.Size([4, 2]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 2]) <----- holder shape
Output: B-P@basicInfo-basicInfoDft
torch.Size([4, 2, 128]) <----- info shape


Input: B-P-EC@BasicInfo-BasicDftGrn_idx
(4, 25, 2)
torch.Size([4, 25, 2]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 25, 2]) <----- holder shape
Output: B-P-EC@BasicInfo-BasicDft
torch.Size([4, 25, 2, 128]) <----- info shape


Input: B-P-EC@DT_min-DTDftGrn_idx
(4, 25, 7)
torch.Size([4, 25, 7]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 25, 7]) <----- holder shape
Output: B-P-EC@DT_min-DTDft
torch.Size([4, 25, 7, 128]) <----- info shape


Input: B-P

# Get the Starting Tensor and Tensor Name

In [ ]:
from fieldnn.utils.dataflowfn import get_dataflow_table, update_df_dataflow

In [ ]:
# full_recfldgrn_EmbedTensor = {}
RECFLD_TO_TENSOR = {}

for full_recfld, info_dict in full_recfldgrn_EmbedTensor.items():
    print(full_recfld, info_dict['info'].shape)
    RECFLD_TO_TENSOR[full_recfld] = info_dict

B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


In [ ]:
[i for i in RECFLD_TO_TENSOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [ ]:
max_layer = max([len(i.split('-')) for i in RECFLD_TO_TENSOR])
max_layer

7

In [ ]:
full_recfldgrn_list = [i for i in RECFLD_TO_TENSOR]
full_recfldgrn_list

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [ ]:
df_dataflow = get_dataflow_table(full_recfldgrn_list)
df_dataflow

,7,6,5,4,3,2
recfldgrn,,,,,,
P@age-AgeNumeDft,NaN,NaN,NaN,NaN,B-P@age-AgeNumeDft,NaN
P@basicInfo-basicInfoDft,NaN,NaN,NaN,NaN,B-P@basicInfo-basicInfoDft,NaN
EC@BasicInfo-BasicDft,NaN,NaN,NaN,B-P-EC@BasicInfo-BasicDft,NaN,NaN
EC@DT_min-DTDft,NaN,NaN,NaN,B-P-EC@DT_min-DTDft,NaN,NaN
A1C@DT-DTDft,NaN,NaN,B-P-EC-A1C@DT-DTDft,NaN,NaN,NaN
A1C@V-A1CNumeDft,NaN,NaN,B-P-EC-A1C@V-A1CNumeDft,NaN,NaN,NaN
Diag@DT-DTDft,NaN,NaN,B-P-EC-Diag@DT-DTDft,NaN,NaN,NaN
Diag@Value-DiagDft,NaN,NaN,B-P-EC-Diag@Value-DiagDft,NaN,NaN,NaN
PNSectSent@Sentence-Tk@TknzLLM,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_dataflow_new = update_df_dataflow(df_dataflow, style = 'ReducerOnly')

df_dataflow_new

,7,6,5,4,3,2
recfldgrn,,,,,,
P@age-AgeNumeDft,NaN,NaN,NaN,NaN,B-P@age-AgeNumeDft,B-P@age
P@basicInfo-basicInfoDft,NaN,NaN,NaN,NaN,B-P@basicInfo-basicInfoDft,B-P@basicInfo
EC@BasicInfo-BasicDft,NaN,NaN,NaN,B-P-EC@BasicInfo-BasicDft,B-P-EC@BasicInfo,B-P@EC@BasicInfo
EC@DT_min-DTDft,NaN,NaN,NaN,B-P-EC@DT_min-DTDft,B-P-EC@DT_min,B-P@EC@DT_min
A1C@DT-DTDft,NaN,NaN,B-P-EC-A1C@DT-DTDft,B-P-EC-A1C@DT,B-P-EC@A1C@DT,B-P@EC@A1C@DT
A1C@V-A1CNumeDft,NaN,NaN,B-P-EC-A1C@V-A1CNumeDft,B-P-EC-A1C@V,B-P-EC@A1C@V,B-P@EC@A1C@V
Diag@DT-DTDft,NaN,NaN,B-P-EC-Diag@DT-DTDft,B-P-EC-Diag@DT,B-P-EC@Diag@DT,B-P@EC@Diag@DT
Diag@Value-DiagDft,NaN,NaN,B-P-EC-Diag@Value-DiagDft,B-P-EC-Diag@Value,B-P-EC@Diag@Value,B-P@EC@Diag@Value
PNSectSent@Sentence-Tk@TknzLLM,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk,B-P-EC-PN-PNSect-PNSectSent,B-P-EC-PN-PNSect,B-P-EC-PN,B-P-EC@PN,B-P@EC@PN


In [ ]:
from fieldnn.utils.dataflowfn import get_NN_List

df_NN = get_NN_List(df_dataflow_new)
df_NN

,SubUnitName,input_names,output_name,input_layerid,output_layerid
0,REDUCER,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6
1,REDUCER,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5
2,REDUCER,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4
3,REDUCER,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4
4,REDUCER,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4
5,REDUCER,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4
6,REDUCER,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4
7,REDUCER,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3
8,REDUCER,[B-P-EC@DT_min-DTDft],B-P-EC@DT_min,4,3
9,REDUCER,[B-P-EC-A1C@DT],B-P-EC@A1C@DT,4,3


# Reducer

## Module

In [ ]:
import torch

# from ..nn.operator.op import ReduceMeanLayer, ReduceSumLayer, RecuderMaxLayer, ConcatenateLayer
from fieldnn.nn.operator.op import ReduceMeanLayer, ReduceSumLayer, RecuderMaxLayer, ConcatenateLayer

class Reducer_Layer(torch.nn.Module):
    def __init__(self, input_fullname, output_fullname, reducer_layer_para):
        super(Reducer_Layer, self).__init__()
        # (bs, xxx, l, c_inp) --> (bs, xxx, c_outp)
        
        # Part 0: Meta
        self.input_fullname = input_fullname
        self.output_fullname = output_fullname
        self.input_size = reducer_layer_para['input_size']
        self.output_size = reducer_layer_para['output_size']

        # Part 1: NN
        nn_name, nn_para = reducer_layer_para[input_fullname]
        if nn_name.lower() == 'mean':
            self.reducer = ReduceMeanLayer()
        elif nn_name.lower() == 'sum':
            self.reducer = ReduceSumLayer()
        elif nn_name.lower() == 'max':
            self.reducer = RecuderMaxLayer()
        elif nn_name.lower() == 'concat':
            self.reducer = ConcatenateLayer()
            # TODO: need to assert something
            assert self.output_size % self.input_size == 0
        else:
            raise ValueError(f'There is no layer for "{nn_name}"')
            
        # Part 3: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in reducer_layer_para['postprocess'].items():
            if method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.output_size, **config)
            
    def forward(self, fullname, holder, info):
        assert self.input_fullname == fullname
        leng_mask = holder == 0
        info = self.reducer(info, leng_mask)
        
        # (3) post-process
        for name, layer in self.postprocess.items():
            info = layer(info)
            
        # fullname = self.fullname
        holder = (leng_mask == 0).sum(-1)
        return self.output_fullname, holder, info

In [ ]:
# from fieldnn.sublayer.reducer import Reducer_Layer

## Config

In [ ]:
from fieldnn.utils.parafn import get_reducer_para

def get_reducer_para(fullname, nn_name, nn_para, input_size, output_size, postprocess):
    reducer_layer_para = {}
    reducer_layer_para['nn_type'] = 'reducer'
    reducer_layer_para['nn_name'] = nn_name
    reducer_layer_para['nn_para'] = nn_para
    reducer_layer_para['input_size'] = input_size
    reducer_layer_para['output_size'] = output_size
    reducer_layer_para['postprocess'] = postprocess
    return reducer_layer_para

## Usage

In [ ]:
# full_recfldgrn_EmbedTensor = {}
RECFLD_TO_TENSOR = {}

for full_recfld, info_dict in full_recfldgrn_EmbedTensor.items():
    print(full_recfld, info_dict['info'].shape)
    RECFLD_TO_TENSOR[full_recfld] = info_dict

B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


In [ ]:
[i for i in RECFLD_TO_TENSOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [ ]:
embed_size

128

In [ ]:
from fieldnn.utils.dataflowfn import get_NN_List

df_NN = get_NN_List(df_dataflow_new)
df_NN

,SubUnitName,input_names,output_name,input_layerid,output_layerid
0,REDUCER,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6
1,REDUCER,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5
2,REDUCER,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4
3,REDUCER,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4
4,REDUCER,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4
5,REDUCER,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4
6,REDUCER,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4
7,REDUCER,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3
8,REDUCER,[B-P-EC@DT_min-DTDft],B-P-EC@DT_min,4,3
9,REDUCER,[B-P-EC-A1C@DT],B-P-EC@A1C@DT,4,3


In [ ]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}

reducer_nnname = 'Max'
default_nnpara = {}
############################



L = []
for idx, nninfo in df_NN.iterrows():
    
    SubUnitName = nninfo['SubUnitName']
    
    if SubUnitName == 'MERGER':
        L.append(nninfo); continue # currently we can ignore the MERGER SubUnit. 
        
    
    # here we want to generate the para_list. 
    input_names = nninfo['input_names']
    assert len(input_names) == 1
    input_name = input_names[0]
    output_name = nninfo['output_name']
    input_layerid = nninfo['output_layerid']

    # The biggest question here is how to generate a useful para_list.
    para_list = []
    for nn_name in [reducer_nnname]: 
        # this will be updated to multiple nn list. currently we only have a reducer
        # the following blocking is for reducer only. 

        nn_name = reducer_nnname
        nn_para = default_nnpara
        input_size = embed_size

        if nn_name.lower() == 'concat':
            # this types of merger only happened in the R in MLRL. 
            fld_childflds = input_name.split('-')[-1]
            assert '@' in fld_childflds
            childflds = fld_childflds.split('@')[-1]
            childflds = childflds.split('&')
            output_size = len(childflds) * embed_size
        else:
            # usally the most case. 
            output_size = embed_size 

        postprocess = process
        para = get_reducer_para(input_name, nn_name, nn_para, input_size, output_size, postprocess)
        para_list.append(para)


    nninfo['para_list'] = para_list
    L.append(nninfo)
        
df_NN_new = pd.DataFrame(L)
df_NN_new
    

,SubUnitName,input_names,output_name,input_layerid,output_layerid,para_list
0,REDUCER,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6,"[{'nn_type': 'reducer', 'nn_name': 'Max', 'nn_..."
1,REDUCER,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5,"[{'nn_type': 'reducer', 'nn_name': 'Max', 'nn_..."
2,REDUCER,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4,"[{'nn_type': 'reducer', 'nn_name': 'Max', 'nn_..."
3,REDUCER,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4,"[{'nn_type': 'reducer', 'nn_name': 'Max', 'nn_..."
4,REDUCER,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4,"[{'nn_type': 'reducer', 'nn_name': 'Max', 'nn_..."
5,REDUCER,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4,"[{'nn_type': 'reducer', 'nn_name': 'Max', 'nn_..."
6,REDUCER,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4,"[{'nn_type': 'reducer', 'nn_name': 'Max', 'nn_..."
7,REDUCER,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3,"[{'nn_type': 'reducer', 'nn_name': 'Max', 'nn_..."
8,REDUCER,[B-P-EC@DT_min-DTDft],B-P-EC@DT_min,4,3,"[{'nn_type': 'reducer', 'nn_name': 'Max', 'nn_..."
9,REDUCER,[B-P-EC-A1C@DT],B-P-EC@A1C@DT,4,3,"[{'nn_type': 'reducer', 'nn_name': 'Max', 'nn_..."


In [ ]:
df_NN_new.columns

Index(['SubUnitName', 'input_names', 'output_name', 'input_layerid',
       'output_layerid', 'para_list'],
      dtype='object')

# Merger

## Module

In [ ]:
import torch
from fieldnn.nn.op import MergerLayer
# from ..nn.op import MergerLayer

class Merger_Layer(torch.nn.Module):
    def __init__(self, input_fullname, output_fullname, merger_layer_para):
        super(Merger_Layer, self).__init__()
        
        # n * (bs, .., c_inp) --> (bs, ..., n, c_inp)
        # Meta Info
        self.input_fullname = input_fullname
        self.output_fullname = output_fullname
        
        # Part 0: sizes
        self.input_size = merger_layer_para['input_size']
        self.output_size = merger_layer_para['output_size']
        
        # Part 1: NN
        nn_name, nn_para = merger_layer_para[input_fullname]
        if nn_name.lower() == 'merger':
            self.Merger = MergerLayer()
        else:
            raise ValueError(f'The NN "{nn_name}" is not available')
        
        # Part 2: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in merger_layer_para['postprocess'].items():
            if method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.output_size, **config)
                
    def forward(self, input_fullname, fullname2data):
        
        fld_list = input_fullname.split('^')
        assert len(fld_list) == len(fullname2data)
        # (1) holder
        holder = self.Merger([data['holder'] for fld, data in fullname2data.items()], -1)
        
        # (2) merge data
        info = self.Merger([data['info'] for fld, data in fullname2data.items()], -2)
        
        # (3) post-process
        for name, layer in self.postprocess.items():
            info = layer(info)
        
        # (4) names
        # prefix = ['-'.join(i.split('-')[:-1]) for i in fullname2data][0]
        # assert prefix == self.fieldname
        # fullname_new = prefix + '2GrnRec2FldType'
        
        return self.output_fullname, holder, info

# SubUnit_Layer

We will keep updating the SubLayer Neural Network

In [ ]:
import torch

class SubUnit_Layer(torch.nn.Module):
    def __init__(self, nninfo):
        super(SubUnit_Layer, self).__init__()
        
        self.nninfo = nninfo
        self.SubUnitName = nninfo['SubUnitName']
        
        # input_names = nninfo['input_names']; assert len(input_names) == 1
        self.input_names = input_names # input_names[0]
        self.output_name = nninfo['output_name']
        
        
        self.LayerDict = torch.nn.ModuleDict()
        para_list = nninfo['para_list']
        for para in para_list:
            nn_type = para['nn_type']
            if nn_type == 'reducer':
                self.layer = Reducer_Layer(**para)
            else:
                raise ValueError(f'For nn_type within SubUnit_Layer, \
                                 {nn_type} is not available')
                

        
    def forward(self, RECFLD_TO_TENSOR):
        
        input_tensor = RECFLD_TO_TENSOR[self.input_name]
        
        # output_tensor = self.layer(self.input_names)
        
        
        assert self.input_fullname == fullname
        leng_mask = holder == 0
        info = self.reducer(info, leng_mask)
        
        # (3) post-process
        for name, layer in self.postprocess.items():
            info = layer(info)
            
        # fullname = self.fullname
        holder = (leng_mask == 0).sum(-1)
        return self.output_fullname, holder, info

# FieldRepr_Layer

In [ ]:
import torch

# from ..nn.operator.op import ReduceMeanLayer, ReduceSumLayer, RecuderMaxLayer, ConcatenateLayer
# from fieldnn.nn.operator.op import ReduceMeanLayer, ReduceSumLayer, RecuderMaxLayer, ConcatenateLayer

class FieldRepr_Layer(torch.nn.Module):
    def __init__(self, df_NN):
        super(FieldRepr_Layer, self).__init__()
        
        self.df_NN = df_NN
        
        self.SubLayerDict = torch.nn.ModuleDict()
        
        for idx, nninfo in df_NN.items():
            output_name = nninfo['output_name']
            para = nninfo['para']
            self.SubLayerDict[output_name] = 
        
    def forward(self, RECFLD_TO_TENSOR):
        
        
        assert self.input_fullname == fullname
        leng_mask = holder == 0
        info = self.reducer(info, leng_mask)
        
        # (3) post-process
        for name, layer in self.postprocess.items():
            info = layer(info)
            
        # fullname = self.fullname
        holder = (leng_mask == 0).sum(-1)
        return self.output_fullname, holder, info

SyntaxError: invalid syntax (223751853.py, line 17)

# --- Parking

In [ ]:
def get_prefix_rec_info(fullname):
    fld = fullname.split('-')[-1]
    if '@' not in fld:
        return fullname
    else:
        singlefld = fld.split('@')[-1]
        rec = fld.replace('@' + singlefld, '')
        return fullname.replace('@' + singlefld, '')

# update the recfld name. 
tmp = pd.DataFrame([{'old': i, 'new': get_prefix_rec_info(i)} for i in RECFLD_TO_TENSOR])
new2old = tmp.groupby('new').apply(lambda x: x['old'].to_list()).to_dict()
for new, old_list in new2old.items():
    
    if len(old_list) == 1 and new != old_list[0]:
        # print(f'Rename a tensor from: \n\t{old_list[0]}, to \n\t{new}')
        RECFLD_TO_TENSOR[new] = RECFLD_TO_TENSOR.pop(old_list[0])
        # RECFLD_TO_TENSOR[new] = RECFLD_TO_TENSOR.get(old_list[0])
    
    elif len(old_list) == 1 and new == old_list[0]:
        continue
    else:
        print('TODO: for the merger layer')
        print(new, old_list)
        

Rename a tensor from: 
	B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM, to 
	B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk


In [ ]:
for full_recfld, info_dict in RECFLD_TO_TENSOR.items():
    print(full_recfld, info_dict['info'].shape)

B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk torch.Size([4, 25, 1, 14, 121, 11, 128])


In [ ]:
RECFLD_TO_REDUCER = {}
NEXT_RECFLD_TO_TENSOR = {}

for fullname in RECFLD_TO_TENSOR:
    if '@' in fullname.split('-')[-1]:
        print('\nSkip this', fullname, '\n')
        continue
        
    input_fullname = fullname
    output_fullname = '-'.join(fullname.split('-')[:-1]) + '@' + fullname.split('-')[- 1]
    # fields = input_fullname.split('-')[-1].split('&')
    # print(input_fullname)
    # print(output_fullname)
    # print(fields)
    
    
    # init the reducer
    #########
    nn_name = 'Max'
    nn_para = {}
    input_size = embed_size
    output_size = embed_size if nn_name != 'concat' else embed_size * 3
    postprocess = process
    #########
    
    reducer_layer_para = get_reducer_para(input_fullname, nn_name, nn_para, input_size, output_size, postprocess)
    # print(reducer_layer_para)
    Reducer = Reducer_Layer(input_fullname, output_fullname, reducer_layer_para)
    
    RECFLD_TO_REDUCER[fullname] = Reducer
    
    
    info_dict = RECFLD_TO_TENSOR[fullname]
    output_fullname, holder_new, info_new = Reducer(input_fullname, 
                                                    info_dict['holder'], 
                                                    info_dict['info'])
    print(input_fullname)
    print(output_fullname)
    print(info_dict['info'].shape)
    print(info_new.shape)
    print('\n')

    NEXT_RECFLD_TO_TENSOR[output_fullname] = {'info': info_new, 'holder': holder_new}
    

B-P@age-AgeNumeDft
B-P@age@AgeNumeDft
torch.Size([4, 19, 128])
torch.Size([4, 128])


B-P@basicInfo-basicInfoDft
B-P@basicInfo@basicInfoDft
torch.Size([4, 2, 128])
torch.Size([4, 128])


B-P-EC@BasicInfo-BasicDft
B-P-EC@BasicInfo@BasicDft
torch.Size([4, 25, 2, 128])
torch.Size([4, 25, 128])


B-P-EC@DT_min-DTDft
B-P-EC@DT_min@DTDft
torch.Size([4, 25, 7, 128])
torch.Size([4, 25, 128])


B-P-EC-A1C@DT-DTDft
B-P-EC-A1C@DT@DTDft
torch.Size([4, 25, 1, 7, 128])
torch.Size([4, 25, 1, 128])


B-P-EC-A1C@V-A1CNumeDft
B-P-EC-A1C@V@A1CNumeDft
torch.Size([4, 25, 1, 37, 128])
torch.Size([4, 25, 1, 128])


B-P-EC-Diag@DT-DTDft
B-P-EC-Diag@DT@DTDft
torch.Size([4, 25, 22, 7, 128])
torch.Size([4, 25, 22, 128])


B-P-EC-Diag@Value-DiagDft
B-P-EC-Diag@Value@DiagDft
torch.Size([4, 25, 22, 3, 128])
torch.Size([4, 25, 22, 128])


B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk
B-P-EC-PN-PNSect-PNSectSent@Sentence@Tk
torch.Size([4, 25, 1, 14, 121, 11, 128])
torch.Size([4, 25, 1, 14, 121, 128])




In [ ]:
for recfld, info_dict in NEXT_RECFLD_TO_TENSOR.items():
    print(recfld, info_dict['info'].shape)

B-P@age@AgeNumeDft torch.Size([4, 128])
B-P-EC-A1C@V@A1CNumeDft torch.Size([4, 25, 1, 128])
B-P-EC-Diag@DT@DTDft torch.Size([4, 25, 22, 128])
B-P-EC-Diag@Value@DiagDft torch.Size([4, 25, 22, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence@Tk torch.Size([4, 25, 1, 14, 121, 128])


In [ ]:
for k, v in RECFLD_TO_TENSOR.items():
    print(k, v.shape)

AttributeError: 'dict' object has no attribute 'shape'

In [ ]:
output_fullname, holder_new, info_new = Reducer(input_fullname, holder, info)
print(output_fullname)
print(holder_new.shape)
print(info_new.shape)

In [ ]:
RECFLD_TO_TENSOR[output_fullname] = info_new

In [ ]:
fullname

In [ ]:
input_fullname = fullname
output_fullname = '-'.join(fullname.split('-')[:-1]) + ':' + fullname.split('-')[- 1]
# fields = fullname_input.split('-')[-1].split('&')
print(input_fullname)
print(output_fullname)
# print(fields)

In [ ]:
#########
nn_name = 'Max'
nn_para = {}
input_size = embed_size
output_size = embed_size if nn_name != 'concat' else embed_size * 3
postprocess = process
#########

reducer_layer_para = get_reducer_para(input_fullname, nn_name, nn_para, input_size, output_size, postprocess)
print(reducer_layer_para)
Reducer = Reducer_Layer(input_fullname, output_fullname, reducer_layer_para)
Reducer

In [ ]:
output_fullname, holder_new, info_new = Reducer(input_fullname, holder, info)
print(output_fullname)
print(holder_new.shape)
print(info_new.shape)

In [ ]:
input_fullname = output_fullname
output_fullname = input_fullname
holder = holder_new
info = info_new


###########
nn_name = 'TFM'
nn_para = {'num_encoder_layers': 6}
input_size = embed_size
output_size = embed_size
embedprocess = process
postprocess = process
###########
learner_layer_para  = get_learner_para(input_fullname, 
                                       nn_name, nn_para, 
                                       input_size, output_size, 
                                       Ignore_PSN_Layers, 
                                       embedprocess, postprocess
                                      )
print(fullname)
print(learner_layer_para)
Learner = Learner_Layer(input_fullname, output_fullname, learner_layer_para)


fullname, holder, info = Learner(input_fullname, holder, info)
print(info.shape)
print(fullname)
print(holder.shape)